In [2]:
import pandas as pd
from pandas_datareader import data as pdr
import FinanceDataReader as fdr
import yfinance
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings("ignore")

In [3]:
etf_tickers = pd.read_csv("ETFs.csv")
stock_tickers = fdr.StockListing("NASDAQ")

100%|██████████| 4618/4618 [00:09<00:00, 500.28it/s]


In [ ]:
def get_position(df):
    window=20
    df["ma"] = df["Adj Close"].rolling(window=window).mean()
    df = df.iloc[window-1:]
    df["deviation"] = df["Adj Close"]-df["ma"]
    df["dev-squared"] = df["deviation"]*df["deviation"]
    df["ma-dev-squared"] = df["dev-squared"].rolling(window=window).mean()
    df= df.iloc[window-1:]
    df["std-dev"] = df["ma-dev-squared"]**(1/2)

    df.loc[df["Adj Close"] <= df["ma"]-2*df["std-dev"], "position"] = -2
    df.loc[(df["ma"]-2*df["std-dev"] < df["Adj Close"]) & (df["Adj Close"] <= df["ma"]), "position"] = -1
    df.loc[(df["ma"] <= df["Adj Close"]) & (df["Adj Close"] < df["ma"]+2*df["std-dev"]), "position"] = 1
    df.loc[df["ma"]+2*df["std-dev"] <= df["Adj Close"], "position"] = 2

    return df

for ticker in tqdm(etf_tickers["Symbol"]):
    df = pdr.get_data_yahoo(ticker, "2022")
    df = get_position(df)
    if df.iloc[-1]["position"] == -2:
        print("BUY :", ticker)
    if df.iloc[-1]["position"] == 2:
        print("SELL :", ticker)
    time.sleep(1)

for ticker in tqdm(stock_tickers["Symbol"]):
    try:
        df = pdr.get_data_yahoo(ticker, "2022")
        df = get_position(df)
        if df.iloc[-1]["position"] == -2:
            print("BUY :", ticker)
    except:
        pass

  6%|▌         | 259/4618 [11:18<3:26:10,  2.84s/it]

BUY : SYNH


 14%|█▍        | 646/4618 [28:18<2:59:23,  2.71s/it]

BUY : OTLY


 14%|█▍        | 656/4618 [28:44<2:58:57,  2.71s/it]

...

 42%|████▏     | 1918/4618 [1:22:37<1:49:22,  2.43s/it]

BUY : HCARU


 49%|████▊     | 2248/4618 [1:36:13<1:40:58,  2.56s/it]

BUY : CFFEU


 49%|████▉     | 2258/4618 [1:36:36<1:31:32,  2.33s/it]